# 유튜브 자막 생성 Application

Colab 환경에서 유튜브 자막 생성 애플리케이션을 만들어봅시다. 애플리케이션 사용자의 유스케이스는 아래와 같습니다.


## 패키지 및 예제 데이터 다운로드하기
python package들을 설치합니다. Colab에서 실행하지 않는 경우 이 셀은 실행하지 않습니다.

In [ ]:
!wget https://raw.githubusercontent.com/mentor1023/dl_apps/main/speech_recognition/requirements-colab.txt
!pip install -r requirements-colab.txt

### OpenAi 버전이 낮아서 아래 API Key 설정시 오류가 발생할 경우 openai 버전을 높여서 설치해야함(아래 설치)
# !pip install --force-reinstall -v openai==1.55.3

In [1]:
!wget https://raw.githubusercontent.com/mentor1023/dl_apps/main/speech_recognition/requirements-colab.txt

--2024-12-12 15:48:19--  https://raw.githubusercontent.com/mentor1023/dl_apps/main/speech_recognition/requirements-colab.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151 [text/plain]
Saving to: ‘requirements-colab.txt’

requirements-colab. 100%[===================>]     151  --.-KB/s    in 0s      

2024-12-12 15:48:19 (1.23 MB/s) - ‘requirements-colab.txt’ saved [151/151]



In [2]:
!pip install -r requirements-colab.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 38.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 61.2 MB/s eta 

In [1]:
!pip install --force-reinstall -v openai==1.55.3

Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
  Obtaining dependency information for openai==1.55.3 from https://files.pythonhosted.org/packages/10/06/691ef3f0112ecf0d7420d0bf35b5d16cf81554141f4b4913a9831031013d/openai-1.55.3-py3-none-any.whl.metadata
  Obtaining dependency information for anyio<5,>=3.5.0 from https://files.pythonhosted.org/packages/a0/7a/4daaf3b6c08ad7ceffea4634ec206faeff697526421c20f07628c7372156/anyio-4.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/2a/39/e50c7c3a983047577ee07d2a9e53faf5a69493943ec3f6a384bdc792deb2/httpx-0.28.1-py3-none-any.whl.metadata
  Obtaining dependency information for jiter<1,>=0.4.0 from https://files.pythonhosted.org/pack

## 패키지 불러오기

In [61]:
import os
import json

import torch
import gradio as gr

from pytubefix import YouTube

from openai import OpenAI
import whisper
from whisper.utils import get_writer

## 애플리케이션 UI 구현하기

In [13]:
with gr.Blocks() as app:
    gr.Markdown("# Youtube 자막 생성기")

    with gr.Row():
        with gr.Column(scale=1):
            link = gr.Textbox(label="Youtube Link")
            subtitle = gr.File(label="Subtitle", file_types=[".srt"])
            transcribe_btn = gr.Button(value="자막 생성!")

        with gr.Column(scale=4):
            output_video = gr.Video(label="Output", height=500)


In [14]:
app.launch(inline=False, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://241eaffabba222d2fb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [15]:
app.close()

Closing server running on port: 7860


## 유튜브 링크에서 영상 가져오기

In [5]:
youtube_link = "https://youtu.be/Or6zvOnSDXA?si=pVZvl2yAc9K8KJm6"

In [ ]:
yt = YouTube(youtube_link)

In [6]:
streams = yt.streams.filter(progressive=True, file_extension="mp4", type="video").order_by("resolution").desc()
for stream in streams:
    print(stream)

<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">


In [7]:
youtube_video_path = "video_from_youtube.mp4"
streams[0].download(filename=youtube_video_path)

'/content/video_from_youtube.mp4'

In [8]:
from IPython.display import HTML
from base64 import b64encode

with open(youtube_video_path,"rb") as f:
    video = f.read()
    data_url = "data:video/mp4;base64," + b64encode(video).decode()

HTML(f"""
<video width=400 controls>
      <source src="{data_url}" type="video/mp4">
</video>
""")

In [9]:
def get_video(link: str):
    video_file_name = "video_from_youtube.mp4"
    yt = YouTube(link)

    # Extract video
    streams = yt.streams.filter(progressive=True, file_extension="mp4", type="video").order_by("resolution").desc()
    streams[0].download(filename=video_file_name)

    return video_file_name

In [10]:
with gr.Blocks() as app:
    gr.Markdown("# Youtube 자막 생성기")

    with gr.Row():
        with gr.Column(scale=1):
            link = gr.Textbox(label="Youtube Link")
            subtitle = gr.File(label="Subtitle", file_types=[".srt"])
            transcribe_btn = gr.Button(value="자막 생성!")

        with gr.Column(scale=4):
            output_video = gr.Video(label="Output", height=500)

    transcribe_btn.click(get_video, [link], [output_video])

In [11]:
app.launch(inline=False, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0061951c5cba2178bf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [12]:
app.close()

Closing server running on port: 7860


## Writer 클래스를 활용해 SRT 자막 파일 만들기 - 라이브러리 활용
ref: https://github.com/openai/whisper/blob/main/whisper/utils.py#L235

In [16]:
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:32<00:00, 95.3MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [17]:
audio_file_name = "audio_from_youtube.webm"

audio_streams = yt.streams.filter(type="audio").order_by("abr").desc()
audio_streams[0].download(filename=audio_file_name)

'/content/audio_from_youtube.webm'

In [18]:
transcript = model.transcribe(audio_file_name)

In [19]:
transcript = model.transcribe(audio_file_name)
transcript

{'text': ' 세상에서 가장 아껴야 할 사람은 너 자신이다. 모든 일을 잘하려고 애쓰지 말 것. 어떤 삶을 살든 사랑만큼은 미루지 말 것. 마음대로 되지 않는 마음은 그냥 쉬게 둘 것. 너무 서두르지 말 것. 그리고 천천히 뜨겁게 살아갈 것. 딸에게 보내는 심리학 편지에서 만날 수 있습니다.',
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 5.46,
   'text': ' 세상에서 가장 아껴야 할 사람은 너 자신이다.',
   'tokens': [50365,
    37990,
    4885,
    20283,
    2216,
    11219,
    1041,
    6612,
    8981,
    12211,
    2124,
    12963,
    31505,
    22158,
    13,
    50638],
   'temperature': 0.0,
   'avg_logprob': -0.2041491176305192,
   'compression_ratio': 1.3943661971830985,
   'no_speech_prob': 0.006797056179493666},
  {'id': 1,
   'seek': 0,
   'start': 6.7,
   'end': 9.16,
   'text': ' 모든 일을 잘하려고 애쓰지 말 것.',
   'tokens': [50700,
    27714,
    7682,
    1638,
    6644,
    8037,
    18914,
    21459,
    49140,
    108,
    1831,
    7058,
    4431,
    13,
    50823],
   'temperature': 0.0,
   'avg_logprob': -0.2041491176305192,
   'compression_ratio': 1.3943661971830985,
   'no_speech_prob': 0.0067

In [20]:
srt_writer = get_writer(output_format="srt", output_dir=".")
srt_writer(transcript, audio_file_name)

In [21]:
!cat audio_from_youtube.srt

1
00:00:00,000 --> 00:00:05,460
세상에서 가장 아껴야 할 사람은 너 자신이다.

2
00:00:06,700 --> 00:00:09,160
모든 일을 잘하려고 애쓰지 말 것.

3
00:00:10,780 --> 00:00:14,720
어떤 삶을 살든 사랑만큼은 미루지 말 것.

4
00:00:16,560 --> 00:00:20,140
마음대로 되지 않는 마음은 그냥 쉬게 둘 것.

5
00:00:21,920 --> 00:00:23,260
너무 서두르지 말 것.

6
00:00:23,260 --> 00:00:27,320
그리고 천천히 뜨겁게 살아갈 것.

7
00:00:27,320 --> 00:00:32,280
딸에게 보내는 심리학 편지에서 만날 수 있습니다.



In [62]:
del model
torch.cuda.empty_cache()

## Writer 클래스를 활용해 SRT 자막 파일 만들기 - API 활용

In [ ]:
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"

In [64]:
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
model_name = "whisper-1"

In [65]:
with open(audio_file_name, "rb") as audio_file:
    transcription = client.audio.transcriptions.create(
        model=model_name,
        file=audio_file,
        response_format="verbose_json"
    )

In [66]:
print(transcription.model_dump_json(indent=2))

{
  "duration": 36.72999954223633,
  "language": "korean",
  "text": "세상에서 가장 아껴야 할 사람은 너 자신이다. 모든 일을 잘하려고 애쓰지 말 것. 어떤 삶을 살든 사랑만큼은 미루지 말 것. 마음대로 되지 않는 마음은 그냥 쉬게 둘 것. 너무 서두르지 말 것. 그리고 천천히 뜨겁게 살아갈 것. 딸에게 보내는 심리학 편지에서 만날 수 있습니다.",
  "segments": [
    {
      "id": 0,
      "avg_logprob": -0.2005208283662796,
      "compression_ratio": 1.3943661451339722,
      "end": 6.0,
      "no_speech_prob": 0.42929473519325256,
      "seek": 0,
      "start": 0.0,
      "temperature": 0.0,
      "text": " 세상에서 가장 아껴야 할 사람은 너 자신이다.",
      "tokens": [
        50364,
        37990,
        4885,
        20283,
        2216,
        11219,
        1041,
        6612,
        8981,
        6606,
        22855,
        2124,
        12963,
        31505,
        22158,
        13,
        50664
      ]
    },
    {
      "id": 1,
      "avg_logprob": -0.2005208283662796,
      "compression_ratio": 1.3943661451339722,
      "end": 10.0,
      "no_speech_prob": 0.42929473519325256,
      "seek": 0,
      "s

In [67]:
srt_writer = get_writer(output_format="srt", output_dir=".")

In [68]:
# srt_writer(dict(transcription), audio_file_name)

'audio_from_youtube.webm'

In [69]:
srt_writer = get_writer(output_format="srt", output_dir=".")
srt_writer(json.loads(transcription.model_dump_json(indent=2)), audio_file_name)

In [70]:
!cat audio_from_youtube.srt

1
00:00:00,000 --> 00:00:06,000
세상에서 가장 아껴야 할 사람은 너 자신이다.

2
00:00:06,000 --> 00:00:10,000
모든 일을 잘하려고 애쓰지 말 것.

3
00:00:10,000 --> 00:00:16,000
어떤 삶을 살든 사랑만큼은 미루지 말 것.

4
00:00:16,000 --> 00:00:21,000
마음대로 되지 않는 마음은 그냥 쉬게 둘 것.

5
00:00:21,000 --> 00:00:24,000
너무 서두르지 말 것.

6
00:00:24,000 --> 00:00:28,000
그리고 천천히 뜨겁게 살아갈 것.

7
00:00:28,000 --> 00:00:33,000
딸에게 보내는 심리학 편지에서 만날 수 있습니다.



## 위스퍼 추론기 구현하기

In [71]:
class WhisperInferencer:
    def __init__(self):
        self.model = whisper.load_model("large")
        self.srt_writer = get_writer(output_format="srt", output_dir=".")

    def inference(self, audio_file_path: str) -> str:
        transcript = self.model.transcribe(audio_file_path)
        self.srt_writer(transcript, audio_file_path)

        filename = os.path.basename(audio_file_path)
        filename = filename.split(".")[0]

        return f"{filename}.srt"

whipser_inferencer = WhisperInferencer()

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [72]:
def transcribe(link: str):
    video_file_name = "video_from_youtube.mp4"
    audio_file_name = "audio_from_youtube.webm"
    yt = YouTube(link)

    # Extract video
    streams = yt.streams.filter(progressive=True, file_extension="mp4", type="video").order_by("resolution").desc()
    streams[0].download(filename=video_file_name)

    # Extract audio
    audio_streams = yt.streams.filter(type="audio").order_by("abr").desc()
    audio_streams[0].download(filename=audio_file_name)

    transcript_file = whipser_inferencer.inference(audio_file_name)
    return transcript_file, [video_file_name, transcript_file]

In [73]:
with gr.Blocks() as app:
    gr.Markdown("# Youtube 자막 생성기")

    with gr.Row():
        with gr.Column(scale=1):
            link = gr.Textbox(label="Youtube Link")
            subtitle = gr.File(label="Subtitle", file_types=[".srt"])
            transcribe_btn = gr.Button(value="자막 생성!")

        with gr.Column(scale=4):
            output_video = gr.Video(label="Output", height=500)

    transcribe_btn.click(transcribe, [link], [subtitle, output_video])

In [74]:
app.launch(inline=False, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7bf7ed3d1c42cfbf0a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [75]:
app.close()

Closing server running on port: 7860


In [ ]:
del whipser_inferencer
torch.cuda.empty_cache()

## 최종 App 구현

In [ ]:
import os

import gradio as gr
import whisper
from pytube import YouTube
from whisper.utils import get_writer

class WhisperInferencer:
    def __init__(self):
        self.model = whisper.load_model("large")
        self.srt_writer = get_writer(output_format="srt", output_dir=".")

    def inference(self, audio_file_path: str) -> str:
        transcript = self.model.transcribe(audio_file_path)
        self.srt_writer(transcript, audio_file_path)

        filename = os.path.basename(audio_file_path)
        filename = filename.split(".")[0]

        return f"{filename}.srt"

whipser_inferencer = WhisperInferencer()

def transcribe(link: str):
    video_file_name = "video_from_youtube.mp4"
    audio_file_name = "audio_from_youtube.webm"
    yt = YouTube(link)

    # Extract video
    streams = yt.streams.filter(progressive=True, file_extension="mp4", type="video").order_by("resolution").desc()
    streams[0].download(filename=video_file_name)

    # Extract audio
    audio_streams = yt.streams.filter(type="audio").order_by("abr").desc()
    audio_streams[0].download(filename=audio_file_name)

    transcript_file = whipser_inferencer.inference(audio_file_name)
    return transcript_file, [video_file_name, transcript_file]

# Set gradio app
with gr.Blocks() as app:
    gr.Markdown("# Youtube 자막 생성기")

    with gr.Row():
        with gr.Column(scale=1):
            link = gr.Textbox(label="Youtube Link")
            subtitle = gr.File(label="Subtitle", file_types=[".srt"])
            transcribe_btn = gr.Button(value="자막 생성!")

        with gr.Column(scale=4):
            output_video = gr.Video(label="Output", height=500)

    transcribe_btn.click(transcribe, [link], [subtitle, output_video])

app.launch(inline=False, share=True)

In [ ]:
app.close()
del whipser_inferencer
torch.cuda.empty_cache()